<a href="https://colab.research.google.com/github/alim98/MPI/blob/main/VGG_Example_FINAL_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Essential downloads

In [1]:
!wget -O downloaded_file.zip "https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"
!wget -O vesicle_cloud__syn_interface__mitochondria_annotation.zip "https://drive.usercontent.google.com/download?id=1qRibZL3kr7MQJQRgDFRquHMQlIGCN4XP&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"

!unzip -q downloaded_file.zip
!unzip -q vesicle_cloud__syn_interface__mitochondria_annotation.zip

--2025-01-27 19:26:35--  https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.143.132, 2a00:1450:4013:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.143.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1264688649 (1.2G) [application/octet-stream]
Saving to: ‘downloaded_file.zip’

downloaded_file.zip 100%[===================>]   1.18G  79.3MB/s    in 14s     

2025-01-27 19:26:52 (85.5 MB/s) - ‘downloaded_file.zip’ saved [1264688649/1264688649]

--2025-01-27 19:26:52--  https://drive.usercontent.google.com/download?id=1qRibZL3kr7MQJQRgDFRquHMQlIGCN4XP&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000
Resolvin

In [2]:

!pip install transformers scikit-learn matplotlib seaborn torch torchvision umap-learn git+https://github.com/funkelab/funlib.learn.torch.git
!pip install openpyxl


  Cloning https://github.com/funkelab/funlib.learn.torch.git to /tmp/pip-req-build-my41fwyp
  Running command git clone --filter=blob:none --quiet https://github.com/funkelab/funlib.learn.torch.git /tmp/pip-req-build-my41fwyp
  Resolved https://github.com/funkelab/funlib.learn.torch.git to commit 049729151c7a2c0320a446dc9d3244ac830f7ea8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.7 MB/s eta 0:00:00
  Created wheel for funlib.learn.torch: filename=funlib.learn.torch-0.1.0-py3-none-any.whl size=13995 sha256=9ab5f7778e28b7f3d5ddb7e5af2d8ee272aed608f417b71fd2155cb69175906d
  Stored in directory: /tmp/pip-ephem-wheel-cache-75z5niev/wheels/ae/7f/7b/ecbd355ccdfbd2bb0ab4f76ea45f60a02a572c88c1f4761e8d
Successfully built funlib.learn.torch


In [3]:

import os
import glob
import imageio.v2 as iio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, Rectangle
from torch.utils.data import Dataset, DataLoader
from transformers import ViTImageProcessor, ViTModel
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
from umap import UMAP
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# GradCam Funcs

## Claudai

In [4]:
# import torch
# import torch.nn.functional as F
# import numpy as np
# import imageio
# import os
# import torch
# import torch.nn.functional as F
# import numpy as np
# import imageio
# import os
# import cv2
# from matplotlib import cm

# class GradCAM3D:
#     def __init__(self, model, target_layer):
#         self.model = model
#         self.target_layer = target_layer
#         self.gradients = None
#         self.activations = None

#         # Register hooks
#         target_layer.register_forward_hook(self.save_activation)
#         target_layer.register_full_backward_hook(self.save_gradient)

#     def save_activation(self, module, input, output):
#         self.activations = output.detach()

#     def save_gradient(self, module, grad_input, grad_output):
#         self.gradients = grad_output[0].detach()

#     def generate_cam(self, input_tensor, target_class=None):
#         # Forward pass
#         model_output = self.model(input_tensor)

#         if target_class is None:
#             target_class = torch.argmax(model_output)

#         # Zero all existing gradients
#         self.model.zero_grad()

#         # Target for backprop
#         one_hot_output = torch.zeros_like(model_output)
#         one_hot_output[0][target_class] = 1

#         # Backward pass
#         model_output.backward(gradient=one_hot_output)

#         # Calculate weights
#         weights = torch.mean(self.gradients, dim=(2, 3, 4))

#         # Generate CAM
#         cam = torch.zeros(self.activations.shape[2:], dtype=torch.float32).to(input_tensor.device)

#         for i, w in enumerate(weights[0]):
#             cam += w * self.activations[0, i]

#         cam = F.relu(cam)  # Apply ReLU to focus on features that have a positive influence

#         # Normalize
#         cam = cam - torch.min(cam)
#         cam = cam / torch.max(cam)

#         return cam

# def apply_colormap(gray_img):
#     """Apply jet colormap to grayscale image."""
#     colored = cm.jet(gray_img)  # Using jet colormap
#     return (colored[:, :, :3] * 255).astype(np.uint8)

# def apply_gradcam(model, dataset, args, num_samples=5):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)
#     model.eval()

#     # Get the last convolutional layer
#     target_layer = None
#     for module in model.features:
#         if isinstance(module, torch.nn.Conv3d):
#             target_layer = module
#     target_layer=model.features[27]
#     print(target_layer)
#     gradcam = GradCAM3D(model, target_layer)

#     for idx in range(min(num_samples, len(dataset))):
#         # Get sample
#         idx=idx+80
#         cube, syn_info, bbox_name = dataset[idx]
#         input_tensor = cube.unsqueeze(0).permute(0, 2, 1, 3, 4).to(device)  # Add batch dimension and adjust channels

#         # Generate CAM
#         cam = gradcam.generate_cam(input_tensor)
#         cam = cam.cpu().numpy()

#         # Create heatmap visualization
#         frames = []
#         original_frames = cube.permute(0, 2, 3, 1).numpy()  # (T, H, W, C)

#         # Ensure we have 80 frames by interpolating if necessary
#         if cam.shape[0] != 80:
#             # Create indices for interpolation
#             old_indices = np.linspace(0, cam.shape[0]-1, cam.shape[0])
#             new_indices = np.linspace(0, cam.shape[0]-1, 80)

#             # Interpolate CAM to 80 frames
#             new_cam = np.zeros((80, cam.shape[1], cam.shape[2]))
#             for i in range(cam.shape[1]):
#                 for j in range(cam.shape[2]):
#                     new_cam[:, i, j] = np.interp(new_indices, old_indices, cam[:, i, j])
#             cam = new_cam

#             # Interpolate original frames if needed
#             if original_frames.shape[0] != 80:
#                 new_original_frames = np.zeros((80, original_frames.shape[1], original_frames.shape[2], original_frames.shape[3]))
#                 for c in range(original_frames.shape[3]):
#                     for i in range(original_frames.shape[1]):
#                         for j in range(original_frames.shape[2]):
#                             new_original_frames[:, i, j, c] = np.interp(new_indices, old_indices, original_frames[:, i, j, c])
#                 original_frames = new_original_frames

#         for t in range(80):  # Process all 80 frames
#             # Normalize frame
#             frame = original_frames[t]
#             frame = (frame - frame.min()) / (frame.max() - frame.min())

#             # Get CAM slice and resize to match frame size
#             cam_slice = cam[t]
#             cam_slice = cv2.resize(cam_slice, (frame.shape[1], frame.shape[0]))

#             # Apply colormap to CAM
#             heatmap = apply_colormap(cam_slice)

#             # Convert original frame to RGB if it's not
#             if frame.shape[-1] == 1:
#                 frame = np.repeat(frame, 3, axis=-1)

#             # Ensure frame is in 0-255 range
#             frame = (frame * 255).astype(np.uint8)

#             # Combine original frame and heatmap
#             alpha = 0.4
#             overlay = cv2.addWeighted(frame, 1 - alpha, heatmap, alpha, 0)
#             frames.append(overlay)

#         # Save as GIF
#         output_dir = os.path.join("Result2", "gradcam")
#         os.makedirs(output_dir, exist_ok=True)
#         output_path = os.path.join(output_dir, f"gradcam_sample_{idx}_type_{args.segmentation_type}.gif")
#         imageio.mimsave(output_path, frames, fps=10)

#         print(f"Saved GradCAM visualization for sample {idx} to {output_path}")

# # Function to run GradCAM analysis
# def run_gradcam_analysis(model, dataset, args):
#     print("Starting GradCAM analysis...")
#     apply_gradcam(model, dataset, args)
#     print("GradCAM analysis completed!")
# # Import the GradCAM implementation

# # After your existing feature extraction code:
# print("Running GradCAM analysis...")
# run_gradcam_analysis(model, dataset, args)

# # Save features as before
# segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")
# # features_df.to_csv(output_csv_path, index=False)

# print("Analysis completed! Check the Result/gradcam directory for visualizations.")


Running GradCAM analysis...


NameError: name 'model' is not defined

In [63]:
# import torch
# import torch.nn.functional as F
# import numpy as np
# import imageio
# import os
# import cv2
# from matplotlib import pyplot as plt

# class GradCAM3D:
#     def __init__(self, model):
#         self.model = model
#         self.gradients = None
#         self.activations = None

#         # Get the last convolutional layer
#         for module in model.modules():
#             if isinstance(module, torch.nn.Conv3d):
#                 self.target_layer = module

#         if self.target_layer is None:
#             raise ValueError("Could not find a convolutional layer")

#         # Register hooks
#         self.target_layer.register_forward_hook(self.forward_hook)
#         self.target_layer.register_forward_hook(lambda m, i, o: setattr(self, 'activations', o.detach()))
#         self.target_layer.register_backward_hook(lambda m, i, o: setattr(self, 'gradients', o[0].detach()))

#     def forward_hook(self, module, input, output):
#         self.activations = output

#     def generate_cam(self, input_tensor, target_class=None):
#         # Ensure model is in eval mode and clear gradients
#         self.model.eval()
#         self.model.zero_grad()

#         # Reshape input tensor for model
#         input_tensor = input_tensor.permute(0, 2, 1, 3, 4)

#         # Forward pass
#         output = self.model(input_tensor)

#         if target_class is None:
#             target_class = output.argmax(dim=1)

#         # Create one-hot encoding for target class
#         target = torch.zeros_like(output, dtype=torch.float32)
#         target[0, target_class] = 1.0

#         # Backward pass
#         output.backward(target, retain_graph=True)

#         # Get gradients and activations
#         gradients = self.gradients
#         activations = self.activations

#         # Calculate weights
#         weights = torch.mean(gradients, dim=(2, 3, 4))[0]

#         # Generate CAM
#         cam = torch.zeros(activations.shape[2:], device=activations.device)
#         for i, w in enumerate(weights):
#             cam += w * activations[0, i]

#         # Apply ReLU and normalize
#         cam = F.relu(cam)
#         cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)

#         return cam.detach().cpu().numpy()

# def apply_gradcam(model, dataset, args, num_samples=5):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)

#     # Initialize GradCAM
#     gradcam = GradCAM3D(model)

#     # Process samples
#     for idx in range(min(num_samples, len(dataset))):
#         # Get input data
#         cube, syn_info, bbox_name = dataset[idx]
#         input_tensor = cube.unsqueeze(0).to(device)

#         # Generate CAM
#         cam = gradcam.generate_cam(input_tensor)

#         # Create visualization frames
#         frames = []
#         original_frames = cube.permute(0, 2, 3, 1).cpu().numpy()

#         for t in range(80):
#             # Get original frame
#             frame = np.squeeze(original_frames[t])
#             frame = (frame - frame.min()) / (frame.max() - frame.min() + 1e-8)

#             # Get CAM slice
#             cam_slice = cv2.resize(cam[t % cam.shape[0]], (frame.shape[1], frame.shape[0]))

#             # Create heatmap
#             heatmap = cv2.applyColorMap(np.uint8(255 * cam_slice), cv2.COLORMAP_JET)
#             heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

#             # Convert frame to RGB
#             frame_rgb = np.uint8(255 * np.stack([frame]*3, axis=-1))

#             # Create overlay
#             alpha = 0.4
#             overlay = cv2.addWeighted(frame_rgb, 1-alpha, heatmap, alpha, 0)
#             frames.append(overlay)

#         # Create output directory
#         output_dir = os.path.join("Result", "gradcam")
#         os.makedirs(output_dir, exist_ok=True)

#         # Save GIF
#         output_path = os.path.join(
#             output_dir,
#             f"gradcam_sample_{idx}_type_{args.segmentation_type}_inputmask_{args.input_mask}.gif"
#         )
#         imageio.mimsave(output_path, frames, fps=10)

#         # Create summary visualization
#         fig, axs = plt.subplots(2, 3, figsize=(15, 10))
#         fig.suptitle(f'GradCAM Visualization - Sample {idx}')

#         # Select middle frame
#         mid_frame = 40

#         # Axial view
#         axs[0, 0].imshow(np.squeeze(original_frames[mid_frame]), cmap='gray')
#         axs[0, 0].set_title('Original')
#         axs[0, 0].axis('off')

#         axs[0, 1].imshow(cam[mid_frame % cam.shape[0]], cmap='jet')
#         axs[0, 1].set_title('GradCAM')
#         axs[0, 1].axis('off')

#         axs[0, 2].imshow(frames[mid_frame])
#         axs[0, 2].set_title('Overlay')
#         axs[0, 2].axis('off')

#         # Sagittal view
#         sagittal_orig = np.squeeze(original_frames[:, :, mid_frame]).mean(axis=2)
#         sagittal_cam = np.mean(cam[:, :, mid_frame % cam.shape[0]], axis=1)

#         axs[1, 0].imshow(sagittal_orig, cmap='gray')
#         axs[1, 0].set_title('Sagittal Original')
#         axs[1, 0].axis('off')

#         axs[1, 1].imshow(sagittal_cam, cmap='jet')
#         axs[1, 1].set_title('Sagittal GradCAM')
#         axs[1, 1].axis('off')

#         # Create sagittal overlay
#         sagittal_heatmap = cv2.applyColorMap(np.uint8(255 * sagittal_cam), cv2.COLORMAP_JET)
#         sagittal_heatmap = cv2.cvtColor(sagittal_heatmap, cv2.COLOR_BGR2RGB)
#         sagittal_frame = np.uint8(255 * np.stack([sagittal_orig]*3, axis=-1))
#         sagittal_overlay = cv2.addWeighted(sagittal_frame, 1-alpha, sagittal_heatmap, alpha, 0)

#         axs[1, 2].imshow(sagittal_overlay)
#         axs[1, 2].set_title('Sagittal Overlay')
#         axs[1, 2].axis('off')

#         # Save summary plot
#         plt.savefig(os.path.join(output_dir, f'gradcam_summary_{idx}.png'))
#         plt.close()

#         print(f"Saved GradCAM visualization for sample {idx}")

# def run_gradcam_analysis(model, dataset, args):
#     print("Starting GradCAM analysis...")
#     apply_gradcam(model, dataset, args)
#     print("GradCAM analysis completed!")
# # After your existing feature extraction code:
# print("Running GradCAM analysis...")
# run_gradcam_analysis(model, dataset, args)



Running GradCAM analysis...
Starting GradCAM analysis...


RuntimeError: Cannot use both regular backward hooks and full backward hooks on a single Module. Please use only one of them.

In [36]:
model.features[27]

Conv3d(96, 192, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))

In [54]:
# import torch
# import torch.nn.functional as F
# import numpy as np
# import imageio
# import os
# import cv2
# from matplotlib import pyplot as plt

# class GradCAM3D:
#     def __init__(self, model, target_layer):
#         self.model = model
#         self.target_layer = target_layer
#         self.activations = None
#         self.gradients = None

#         # Register hooks
#         self.target_layer.register_forward_hook(self._save_activation)
#         self.target_layer.register_full_backward_hook(self._save_gradient)

#     def _save_activation(self, module, input, output):
#         self.activations = output

#     def _save_gradient(self, module, grad_input, grad_output):
#         self.gradients = grad_output[0]

#     def generate_cam(self, input_tensor, target_class=None):
#         # Ensure model is in eval mode
#         self.model.eval()

#         # Forward pass to get class predictions
#         pred = self.model(input_tensor)

#         if target_class is None:
#             target_class = pred.argmax(dim=1)

#         # Zero gradients
#         self.model.zero_grad()

#         # Target for backprop
#         one_hot = torch.zeros_like(pred)
#         one_hot[0][target_class] = 1

#         # Backward pass
#         pred.backward(gradient=one_hot, retain_graph=True)

#         # Get weights
#         weights = torch.mean(self.gradients, dim=(2, 3, 4))

#         # Generate CAM
#         batch_size = self.activations.shape[0]
#         cam = torch.zeros(self.activations.shape[2:], device=self.activations.device)

#         # Weighted sum of activation maps
#         for w, act in zip(weights[0], self.activations[0]):
#             cam += w * act

#         # Apply ReLU
#         cam = F.relu(cam)

#         # Normalize
#         cam = cam - cam.min()
#         cam = cam / (cam.max() + 1e-7)

#         return cam.detach().cpu().numpy()

# def apply_gradcam(model, dataset, args, num_samples=5, layer_name='conv3d_18'):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = model.to(device)

#     # Get target layer
#     target_layer = None
#     for name, module in model.named_modules():
#         if isinstance(module, torch.nn.Conv3d) and name == layer_name:
#             target_layer = module
#             break

#     # if target_layer is None:
#     #     raise ValueError(f"Could not find layer {layer_name}")
#     target_layer=model.features[27]

#     gradcam = GradCAM3D(model, target_layer)

#     for idx in range(min(num_samples, len(dataset))):
#         # Get input data
#         cube, syn_info, bbox_name = dataset[idx]
#         input_tensor = cube.unsqueeze(0).to(device)

#         # Generate GradCAM
#         cam = gradcam.generate_cam(input_tensor)

#         # Create visualization
#         frames = []
#         original_frames = cube.permute(0, 2, 3, 1).cpu().numpy()

#         for t in range(80):  # Process all 80 frames
#             # Get original frame
#             frame = original_frames[t]
#             frame = (frame - frame.min()) / (frame.max() - frame.min() + 1e-7)

#             # Get CAM slice and resize
#             if t < cam.shape[0]:
#                 cam_slice = cam[t]
#             else:
#                 # For frames beyond CAM depth, use the last available slice
#                 cam_slice = cam[-1]

#             # Resize CAM to match frame size
#             cam_slice = cv2.resize(cam_slice, (frame.shape[1], frame.shape[0]))

#             # Apply colormap
#             heatmap = cv2.applyColorMap(np.uint8(255 * cam_slice), cv2.COLORMAP_JET)
#             heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

#             # Convert grayscale frame to RGB
#             frame_rgb = np.uint8(255 * np.stack([frame]*3, axis=-1))

#             # Create overlay
#             alpha = 0.4
#             overlay = cv2.addWeighted(frame_rgb, 1-alpha, heatmap, alpha, 0)
#             frames.append(overlay)

#         # Save GIF
#         output_dir = os.path.join("Result", "gradcam")
#         os.makedirs(output_dir, exist_ok=True)

#         output_path = os.path.join(
#             output_dir,
#             f"gradcam_sample_{idx}_type_{args.segmentation_type}_inputmask_{args.input_mask}.gif"
#         )

#         # Save with consistent frame rate
#         imageio.mimsave(output_path, frames, fps=10)

#         # Create and save summary visualization
#         fig, axs = plt.subplots(2, 3, figsize=(15, 10))
#         fig.suptitle(f'GradCAM Visualization - Sample {idx}')

#         # Select representative slices
#         mid_frame = 40  # Middle frame

#         # Axial view
#         axs[0, 0].imshow(original_frames[mid_frame], cmap='gray')
#         axs[0, 0].set_title('Original')
#         axs[0, 0].axis('off')

#         axs[0, 1].imshow(cam[mid_frame], cmap='jet')
#         axs[0, 1].set_title('GradCAM')
#         axs[0, 1].axis('off')

#         axs[0, 2].imshow(frames[mid_frame])
#         axs[0, 2].set_title('Overlay')
#         axs[0, 2].axis('off')

#         # Sagittal view
#         sagittal_orig = original_frames[:, :, mid_frame].mean(axis=2)
#         sagittal_cam = cam[:, :, mid_frame]

#         axs[1, 0].imshow(sagittal_orig, cmap='gray')
#         axs[1, 0].set_title('Sagittal Original')
#         axs[1, 0].axis('off')

#         axs[1, 1].imshow(sagittal_cam, cmap='jet')
#         axs[1, 1].set_title('Sagittal GradCAM')
#         axs[1, 1].axis('off')

#         # Create sagittal overlay
#         sagittal_heatmap = cv2.applyColorMap(np.uint8(255 * sagittal_cam), cv2.COLORMAP_JET)
#         sagittal_heatmap = cv2.cvtColor(sagittal_heatmap, cv2.COLOR_BGR2RGB)
#         sagittal_frame = np.uint8(255 * np.stack([sagittal_orig]*3, axis=-1))
#         sagittal_overlay = cv2.addWeighted(sagittal_frame, 1-alpha, sagittal_heatmap, alpha, 0)

#         axs[1, 2].imshow(sagittal_overlay)
#         axs[1, 2].set_title('Sagittal Overlay')
#         axs[1, 2].axis('off')

#         # Save plot
#         plt.savefig(os.path.join(output_dir, f'gradcam_summary2_{idx}.png'))
#         plt.close()

#         print(f"Saved GradCAM visualization for sample {idx}")

# def run_gradcam_analysis(model, dataset, args):
#     print("Starting GradCAM analysis...")
#     apply_gradcam(model, dataset, args)
#     print("GradCAM analysis completed!")

# # Import the GradCAM implementation

# # After your feature extraction and before saving the features:
# print("Running GradCAM analysis...")
# run_gradcam_analysis(model, dataset, args)

# # # Continue with your existing feature saving code
# # print("Saving features...")
# # segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# # output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")
# # features_df.to_csv(output_csv_path, index=False)

# print("Analysis completed! Check the Result/gradcam directory for visualizations.")

Running GradCAM analysis...
Starting GradCAM analysis...


RuntimeError: Given groups=1, weight of size [24, 1, 3, 3, 3], expected input[1, 80, 1, 80, 80] to have 1 channels, but got 80 channels instead

# 1. Visualizing Feature Maps (Activations)


To understand where model attends to in terms of feature extraction, we can visualize the feature maps from different layers of the model. The activations represent which regions of the 3D input are being activated by different filters at each convolutional layer.

In [7]:

args = parse_args()
args.input_mask=False
args.segmentation_type=0
# Initialize the arguments (or use your existing args)
# Use the main function to load data and prepare the dataset
# cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=args.segmentation_type,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=False
)

In [16]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from tqdm import tqdm

def visualize_sample_attention(model, dataset, sample_idx=0, output_dir="attention_gifs", layer_index=30):
    """
    Visualize attention for a specific sample from your dataset
    Returns GIF path and metadata for the visualized sample
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()
    os.makedirs(output_dir, exist_ok=True)

    # Get sample from your dataset
    pixel_values, syn_info, bbox_name = dataset[sample_idx]

    # Convert to model input format
    input_tensor = pixel_values.permute(1, 0, 2, 3).unsqueeze(0).to(device)  # [1, C, D, H, W]

    # Setup activation hook
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    hook = model.features[layer_index].register_forward_hook(get_activation('conv'))

    # Forward pass
    with torch.no_grad():
        _ = model.features(input_tensor)

    hook.remove()

    # Process activations
    activations = activation['conv'][0].cpu().numpy()  # [C, D, H, W]
    attention_map = np.mean(activations, axis=0)  # Average across channels
    attention_map = (attention_map - attention_map.min()) / (attention_map.max() - attention_map.min())

    # Get original data from dataset
    original_vol = pixel_values.numpy().transpose(1, 0, 2, 3)  # [C, D, H, W]

    # Resize attention map to match original volume if needed
    if attention_map.shape != original_vol.shape[1:]:
        zoom_factors = [
            original_vol.shape[1]/attention_map.shape[0],
            original_vol.shape[2]/attention_map.shape[1],
            original_vol.shape[3]/attention_map.shape[2]
        ]
        attention_map = ndimage.zoom(attention_map, zoom_factors, order=1)

    # Create overlay frames
    frames = []
    for z in range(original_vol.shape[1]):
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))

        # Original slice
        ax[0].imshow(original_vol[0, z], cmap='gray')
        ax[0].set_title(f'Original Slice Z{z}\n{bbox_name}')
        ax[0].axis('off')

        # Overlay attention
        ax[1].imshow(original_vol[0, z], cmap='gray')
        im = ax[1].imshow(attention_map[z], cmap='jet', alpha=0.5)
        plt.colorbar(im, ax=ax[1], fraction=0.046, pad=0.04)
        ax[1].set_title(f'Attention Map (Layer {layer_index})\n{syn_info["Var1"]}')
        ax[1].axis('off')

        # Save frame
        fig.canvas.draw()
        frame = np.array(fig.canvas.renderer._renderer)
        frames.append(frame)
        plt.close(fig)

    # Save GIF with metadata in filename
    gif_name = f"{bbox_name}_syn{syn_info['Var1']}_layer{layer_index}.gif"
    gif_path = os.path.join(output_dir, gif_name)
    imageio.mimsave(gif_path, frames, duration=0.2)

    return gif_path, syn_info

# def visualize_multiple_samples(model, dataset, num_samples=5):
#     """Visualize attention for multiple random samples"""
#     results = []
#     for _ in tqdm(range(num_samples)):
#         # Random sample respecting bbox boundaries
#         sample_idx = np.random.randint(0, len(dataset))
#         gif_path, syn_info = visualize_sample_attention(model, dataset, sample_idx)
#         results.append((gif_path, syn_info))
#     return results

# Visualize first 5 samples
# results = visualize_multiple_samples(model, dataset, num_samples=5)

# Print results
for gif_path, syn_info in results:
    print(f"Generated visualization for {syn_info['bbox_name']} synapse {syn_info['Var1']}")
    print(f"GIF saved at: {gif_path}\n")

100%|██████████| 5/5 [00:57<00:00, 11.40s/it]

Generated visualization for bbox1 synapse non_spine_synapse_023
GIF saved at: attention_gifs/bbox1_synnon_spine_synapse_023_layer30.gif

Generated visualization for bbox3 synapse non_spine_synapse_005
GIF saved at: attention_gifs/bbox3_synnon_spine_synapse_005_layer30.gif

Generated visualization for bbox1 synapse non_spine_synapse_052
GIF saved at: attention_gifs/bbox1_synnon_spine_synapse_052_layer30.gif

Generated visualization for bbox1 synapse non_spine_synapse_048
GIF saved at: attention_gifs/bbox1_synnon_spine_synapse_048_layer30.gif

Generated visualization for bbox5 synapse non_spine_synapse_011
GIF saved at: attention_gifs/bbox5_synnon_spine_synapse_011_layer30.gif



In [17]:

def visualize_multiple_samples(model, dataset, num_samples=5, layer_index=33):
    """Visualize attention for multiple random samples with layer index"""
    results = []
    for _ in tqdm(range(num_samples)):
        sample_idx = np.random.randint(0, len(dataset))
        gif_path, syn_info = visualize_sample_attention(
            model, dataset, sample_idx, layer_index=layer_index
        )
        results.append((gif_path, syn_info))
    return results

# ... [Your existing dataset preparation code] ...

# Main analysis loops
def run_visualization_analysis(model, args, vol_data_dict, syn_df, processor):
    # Original dataset configuration
    base_dataset = VideoMAEDataset(
        vol_data_dict=vol_data_dict,
        synapse_df=syn_df,
        processor=processor,
        segmentation_type=args.segmentation_type,
        subvol_size=args.subvol_size,
        num_frames=args.num_frames,
        alpha=args.alpha,
        input_mask=args.input_mask
    )

    # 1. Different segmentation types
    for seg_type in [0, 1, 2, 3, 4, 5]:
        args.segmentation_type = seg_type
        seg_dataset = VideoMAEDataset(
            vol_data_dict=vol_data_dict,
            synapse_df=syn_df,
            processor=processor,
            segmentation_type=seg_type,
            subvol_size=args.subvol_size,
            num_frames=args.num_frames,
            alpha=args.alpha,
            input_mask=args.input_mask
        )
        print(f"\nVisualizing segmentation type {seg_type}")
        _ = visualize_multiple_samples(model, seg_dataset, num_samples=3)

    # 2. Input masking comparison
    for use_mask in [True, False]:
        args.input_mask = use_mask
        mask_dataset = VideoMAEDataset(
            vol_data_dict=vol_data_dict,
            synapse_df=syn_df,
            processor=processor,
            segmentation_type=args.segmentation_type,
            subvol_size=args.subvol_size,
            num_frames=args.num_frames,
            alpha=args.alpha,
            input_mask=use_mask
        )
        print(f"\nVisualizing with input_mask={use_mask}")
        _ = visualize_multiple_samples(model, mask_dataset, num_samples=3)

    # 3. Different layer indices
    for layer_idx in [12, 19, 26, 33]:
        print(f"\nVisualizing layer {layer_idx}")
        _ = visualize_multiple_samples(model, base_dataset, num_samples=3, layer_index=layer_idx)

# Run the analysis
run_visualization_analysis(model, args, vol_data_dict, syn_df, processor)


Visualizing segmentation type 0


100%|██████████| 3/3 [00:34<00:00, 11.65s/it]



Visualizing segmentation type 1


100%|██████████| 3/3 [00:32<00:00, 10.93s/it]



Visualizing segmentation type 2


...

100%|██████████| 3/3 [00:37<00:00, 12.35s/it]



Visualizing layer 26


100%|██████████| 3/3 [00:30<00:00, 10.24s/it]



Visualizing layer 33


100%|██████████| 3/3 [00:30<00:00, 10.29s/it]


In [27]:
# prompt: download /content/attention_gifs this folder

import shutil
import os

# Create a zip archive of the folder
shutil.make_archive("attention_gifs", 'zip', "/content/attention_gifs6")

# # Download the zip archive
from google.colab import files
files.download("attention_gifs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:

!cp attention_gifs.zip /content/drive/MyDrive/

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## fixed samples final grad

In [26]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from tqdm import tqdm
from collections import defaultdict

def visualize_sample_attention(model, dataset, sample_idx=0, output_dir="attention_gifs6", layer_index=30):
    """
    Visualize attention for a specific sample with metadata in title/filename
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()
    os.makedirs(output_dir, exist_ok=True)

    # Get sample data
    pixel_values, syn_info, bbox_name = dataset[sample_idx]

    # Forward pass setup
    input_tensor = pixel_values.permute(1, 0, 2, 3).unsqueeze(0).to(device)
    activation = {}

    # Hook setup
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    hook = model.features[layer_index].register_forward_hook(get_activation('conv'))

    # Forward pass
    with torch.no_grad():
        _ = model.features(input_tensor)
    hook.remove()

    # Process activations
    activations = activation['conv'][0].cpu().numpy()
    attention_map = np.mean(activations, axis=0)
    attention_map = (attention_map - attention_map.min()) / (attention_map.max() - attention_map.min())

    # Get dataset parameters
    seg_type = dataset.segmentation_type
    use_mask = dataset.input_mask
    original_vol = pixel_values.numpy().transpose(1, 0, 2, 3)

    # Resize attention map if needed
    if attention_map.shape != original_vol.shape[1:]:
        zoom_factors = [
            original_vol.shape[1]/attention_map.shape[0],
            original_vol.shape[2]/attention_map.shape[1],
            original_vol.shape[3]/attention_map.shape[2]
        ]
        attention_map = ndimage.zoom(attention_map, zoom_factors, order=1)

    # Create frames with metadata
    frames = []
    for z in range(original_vol.shape[1]):
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))

        # Original slice
        ax[0].imshow(original_vol[0, z], cmap='gray')
        ax[0].set_title(f'Original Slice Z{z}\n{bbox_name}')
        ax[0].axis('off')

        # Attention overlay
        ax[1].imshow(original_vol[0, z], cmap='gray')
        im = ax[1].imshow(attention_map[z], cmap='jet', alpha=0.5)
        plt.colorbar(im, ax=ax[1], fraction=0.046, pad=0.04)
        title = (
            f"Attention Map (Layer {layer_index}, Seg {seg_type}, Mask {use_mask})\n"
            f"{syn_info['Var1']}"
        )
        ax[1].set_title(title)
        ax[1].axis('off')

        # Save frame
        fig.canvas.draw()
        frames.append(np.array(fig.canvas.renderer._renderer))
        plt.close(fig)

    # Save GIF with comprehensive filename
    gif_name = (
        f"{bbox_name}_syn{syn_info['Var1']}_"
        f"layer{layer_index}_seg{seg_type}_mask{use_mask}.gif"
    )
    gif_path = os.path.join(output_dir, gif_name)
    imageio.mimsave(gif_path, frames, duration=0.2)

    return gif_path, syn_info

# def visualize_multiple_samples(model, dataset, num_samples=5, layer_index=30):
#     """Visualize samples ensuring one per B-box"""
#     results = []
#     bbox_list = ['bbox1','bbox2','bbox3','bbox4','bbox5','bbox6', ]
#     np.random.shuffle(bbox_list)

#     for bbox_name in bbox_list[:num_samples]:
#         indices = dataset.bbox_to_indices[bbox_name]
#         sample_idx = np.random.choice(indices)
#         gif_path, syn_info = visualize_sample_attention(
#             model, dataset, sample_idx, layer_index=layer_index
#         )
#         results.append((gif_path, syn_info))

#     return results
# def visualize_multiple_samples(model, dataset, num_samples=5, layer_index=30):
#     """Visualize samples ensuring one per B-box"""
#     results = []
#     bbox_list = ['bbox1', 'bbox2', 'bbox3', 'bbox4', 'bbox5', 'bbox6']
#     np.random.shuffle(bbox_list)

#     for bbox_name in tqdm(bbox_list[:num_samples]):
#         # Instead of looking up indices, we'll find samples that match the bbox_name
#         sample_indices = []
#         for i in range(len(dataset)):
#             _, _, sample_bbox_name = dataset[i]
#             if sample_bbox_name == bbox_name:
#                 sample_indices.append(i)

#         if sample_indices:
#             sample_idx = np.random.choice(sample_indices)
#             gif_path, syn_info = visualize_sample_attention(
#                 model, dataset, sample_idx, layer_index=layer_index
#             )
#             results.append((gif_path, syn_info))
#         else:
#             print(f"No samples found for {bbox_name}")

#     return results
# sample_idx = np.random.randint(0, len(dataset))
# sample_idx2 = np.random.randint(0, len(dataset))
# sample_idx3 = np.random.randint(0, len(dataset))
sample_idx_list = [np.random.randint(0, len(dataset)) for _ in range(3)]
def visualize_multiple_samples(model, dataset,  layer_index=33,sample_idx=sample_idx_list):
    """Visualize attention for multiple random samples without B-box sampling"""
    results = []
    for i in tqdm(range(len(sample_idx_list))):
        # Randomly select a sample index
        # sample_idx = np.random.randint(0, len(dataset))
        gif_path, syn_info = visualize_sample_attention(
            model, dataset, sample_idx_list[i], layer_index=layer_index
        )
        results.append((gif_path, syn_info))
    return results

def run_visualization_analysis(model, args, vol_data_dict, syn_df, processor):
    """Main analysis with correct parameter combinations"""
    # Segmentation types 0-5 with mask=False
    for seg_type in [0, 3, 4, 5]:
        dataset = VideoMAEDataset(
            vol_data_dict=vol_data_dict,
            synapse_df=syn_df,
            processor=processor,
            segmentation_type=seg_type,
            subvol_size=args.subvol_size,
            num_frames=args.num_frames,
            alpha=args.alpha,
            input_mask=False
        )
        print(f"\nVisualizing seg_type={seg_type}, mask=False")
        _ = visualize_multiple_samples(model, dataset)

    # Segmentation types 1-5 with mask=True
    for seg_type in [1,  4, 5]:
        dataset = VideoMAEDataset(
            vol_data_dict=vol_data_dict,
            synapse_df=syn_df,
            processor=processor,
            segmentation_type=seg_type,
            subvol_size=args.subvol_size,
            num_frames=args.num_frames,
            alpha=args.alpha,
            input_mask=True
        )
        print(f"\nVisualizing seg_type={seg_type}, mask=True")
        _ = visualize_multiple_samples(model, dataset)

    # Layer analysis with base configuration
    base_dataset = VideoMAEDataset(
        vol_data_dict=vol_data_dict,
        synapse_df=syn_df,
        processor=processor,
        segmentation_type=args.segmentation_type,
        subvol_size=args.subvol_size,
        num_frames=args.num_frames,
        alpha=args.alpha,
        input_mask=args.input_mask
    )
    for layer_idx in [12, 19, 26, 33]:
        print(f"\nVisualizing layer {layer_idx}")
        _ = visualize_multiple_samples(model, base_dataset, layer_index=layer_idx)

run_visualization_analysis(model, args, vol_data_dict, syn_df, processor)


Visualizing seg_type=0, mask=False


100%|██████████| 3/3 [00:45<00:00, 15.31s/it]



Visualizing seg_type=3, mask=False


100%|██████████| 3/3 [00:36<00:00, 12.19s/it]



Visualizing seg_type=4, mask=False


...

100%|██████████| 3/3 [00:38<00:00, 12.99s/it]



Visualizing layer 26


100%|██████████| 3/3 [00:45<00:00, 15.02s/it]



Visualizing layer 33


100%|██████████| 3/3 [00:38<00:00, 12.91s/it]


# VGG Example


#1Data

Just Uncomment either 1.1 or 1.2 for data loading

## 1.1 Dark background (sectoins that have not segmentaions be 0)

In [4]:
import os
import glob
import io
import argparse
import multiprocessing
from typing import List, Tuple

import numpy as np
import pandas as pd
import imageio.v3 as iio
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
class SimpleVideoProcessor:
    def __init__(self, size=(80, 80), mean=(0.485,),  # Use a single channel for grayscale
                 std=(0.229,)):
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(size),
            transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

    def __call__(self, frames, return_tensors=None):
        processed_frames = [self.transform(frame) for frame in frames]
        pixel_values = torch.stack(processed_frames)
        if return_tensors == "pt":
            return {"pixel_values": pixel_values}
        else:
            return pixel_values


def load_volumes(bbox_name: str, raw_base_dir: str, seg_base_dir: str, add_mask_base_dir: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    raw_dir = os.path.join(raw_base_dir, bbox_name)
    seg_dir = os.path.join(seg_base_dir, bbox_name)

    if bbox_name.startswith("bbox"):
        bbox_num = bbox_name.replace("bbox", "")
        add_mask_dir = os.path.join(add_mask_base_dir, f"bbox_{bbox_num}")
    else:
        add_mask_dir = os.path.join(add_mask_base_dir, bbox_name)

    raw_tif_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
    seg_tif_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))
    add_mask_tif_files = sorted(glob.glob(os.path.join(add_mask_dir, 'slice_*.tif')))

    if not (len(raw_tif_files) == len(seg_tif_files) == len(add_mask_tif_files)):
        return None, None, None

    try:
        raw_vol = np.stack([iio.imread(f) for f in raw_tif_files], axis=0)
        seg_vol = np.stack([iio.imread(f).astype(np.uint32) for f in seg_tif_files], axis=0)
        add_mask_vol = np.stack([iio.imread(f).astype(np.uint32) for f in add_mask_tif_files], axis=0)
        return raw_vol, seg_vol, add_mask_vol
    except Exception as e:
        return None, None, None

def create_segmented_cube(
    raw_vol: np.ndarray,
    seg_vol: np.ndarray,
    add_mask_vol: np.ndarray,
    central_coord: Tuple[int, int, int],
    side1_coord: Tuple[int, int, int],
    side2_coord: Tuple[int, int, int],
    segmentation_type: int,
    subvolume_size: int = 80,
    alpha: float = 0.3,
    input_mask: bool = False  # New parameter
) -> np.ndarray:
    def create_segment_masks(segmentation_volume, s1_coord, s2_coord):
        x1, y1, z1 = s1_coord
        x2, y2, z2 = s2_coord
        seg_id_1 = segmentation_volume[z1, y1, x1]
        seg_id_2 = segmentation_volume[z2, y2, x2]

        mask_1 = (segmentation_volume == seg_id_1) if seg_id_1 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
        mask_2 = (segmentation_volume == seg_id_2) if seg_id_2 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
        return mask_1, mask_2

    mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)
    mask_3_full = (add_mask_vol > 0)

    half_size = subvolume_size // 2
    cx, cy, cz = central_coord
    x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
    y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
    z_start, z_end = max(cz - half_size, 0), min(cz + half_size, raw_vol.shape[0])

    sub_raw = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_3 = mask_3_full[z_start:z_end, y_start:y_end, x_start:x_end]

    pad_z = subvolume_size - sub_raw.shape[0]
    pad_y = subvolume_size - sub_raw.shape[1]
    pad_x = subvolume_size - sub_raw.shape[2]

    if pad_z > 0 or pad_y > 0 or pad_x > 0:
        sub_raw = np.pad(sub_raw, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=0)
        sub_mask_1 = np.pad(sub_mask_1, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_2 = np.pad(sub_mask_2, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_3 = np.pad(sub_mask_3, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)

    sub_raw = sub_raw[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_1 = sub_mask_1[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_2 = sub_mask_2[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_3 = sub_mask_3[:subvolume_size, :subvolume_size, :subvolume_size]

    overlaid_cube = np.zeros((subvolume_size, subvolume_size, 3, subvolume_size), dtype=np.uint8)

    for z in range(subvolume_size):
        raw_slice = sub_raw[z].astype(np.float32)
        mn, mx = raw_slice.min(), raw_slice.max()
        if mx > mn:
            raw_slice = (raw_slice - mn) / (mx - mn)
        else:
            raw_slice = raw_slice - mn

        if input_mask:  # New masking logic
            combined_mask = np.zeros_like(raw_slice, dtype=np.float32)
            if segmentation_type in [1, 3, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_1[z])
            if segmentation_type in [2, 3, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_2[z])
            if segmentation_type in [4, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_3[z])

            masked_raw = raw_slice * combined_mask
            masked_rgb = np.stack([masked_raw]*3, axis=-1)
            overlaid_image = (masked_rgb * 255).astype(np.uint8)
        else:  # Original overlay logic
            raw_rgb = np.stack([raw_slice]*3, axis=-1)
            mask1_rgb = np.zeros_like(raw_rgb)
            mask2_rgb = np.zeros_like(raw_rgb)
            mask3_rgb = np.zeros_like(raw_rgb)

            if segmentation_type in [1, 3, 5]:
                mask1_rgb[sub_mask_1[z]] = [1, 0, 0]
            if segmentation_type in [2, 3, 5]:
                mask2_rgb[sub_mask_2[z]] = [0, 0, 1]
            if segmentation_type in [4, 5]:
                mask3_rgb[sub_mask_3[z]] = [0, 1, 0]

            combined_masks = mask1_rgb + mask2_rgb + mask3_rgb
            combined_masks = np.clip(combined_masks, 0, 1)
            overlaid_image = (1 - alpha) * raw_rgb + alpha * combined_masks
            overlaid_image = (np.clip(overlaid_image, 0, 1) * 255).astype(np.uint8)

        overlaid_cube[:, :, :, z] = overlaid_image

    return overlaid_cube

class VideoMAEDataset(Dataset):
    def __init__(self, vol_data_dict: dict, synapse_df: pd.DataFrame, processor,
                 segmentation_type: int, subvol_size: int = 80, num_frames: int = 16,
                 alpha: float = 0.3, input_mask: bool = False):  # Added input_mask
        self.vol_data_dict = vol_data_dict
        self.synapse_df = synapse_df.reset_index(drop=True)
        self.processor = processor
        self.segmentation_type = segmentation_type
        self.subvol_size = subvol_size
        self.num_frames = num_frames
        self.alpha = alpha
        self.input_mask = input_mask  # Store input_mask flag

    def __len__(self):
        return len(self.synapse_df)

    def __getitem__(self, idx):
        syn_info = self.synapse_df.iloc[idx]
        bbox_name = syn_info['bbox_name']
        raw_vol, seg_vol, add_mask_vol = self.vol_data_dict.get(bbox_name, (None, None, None))

        if raw_vol is None:
            return torch.zeros((self.num_frames, 3, self.subvol_size, self.subvol_size), dtype=torch.float32), syn_info, bbox_name

        central_coord = (int(syn_info['central_coord_1']), int(syn_info['central_coord_2']), int(syn_info['central_coord_3']))
        side1_coord = (int(syn_info['side_1_coord_1']), int(syn_info['side_1_coord_2']), int(syn_info['side_1_coord_3']))
        side2_coord = (int(syn_info['side_2_coord_1']), int(syn_info['side_2_coord_2']), int(syn_info['side_2_coord_3']))

        overlaid_cube = create_segmented_cube(
            raw_vol=raw_vol,
            seg_vol=seg_vol,
            add_mask_vol=add_mask_vol,
            central_coord=central_coord,
            side1_coord=side1_coord,
            side2_coord=side2_coord,
            segmentation_type=self.segmentation_type,
            subvolume_size=self.subvol_size,
            alpha=self.alpha,
            input_mask=self.input_mask  # Pass the flag
        )

        frames = [overlaid_cube[..., z] for z in range(overlaid_cube.shape[3])]
        if len(frames) < self.num_frames:
            frames += [frames[-1]] * (self.num_frames - len(frames))
        elif len(frames) > self.num_frames:
            indices = np.linspace(0, len(frames)-1, self.num_frames, dtype=int)
            frames = [frames[i] for i in indices]

        inputs = self.processor(frames, return_tensors="pt")
        return inputs["pixel_values"].squeeze(0).float(), syn_info, bbox_name

def parse_args():
    parser = argparse.ArgumentParser(description="VideoMAE Pre-training Script with Segmented Videos and Additional Masks")
    parser.add_argument('--raw_base_dir', type=str, default='raw')
    parser.add_argument('--seg_base_dir', type=str, default='seg')
    parser.add_argument('--add_mask_base_dir', type=str, default='')
    parser.add_argument('--bbox_name', type=str, default=['bbox1','bbox2','bbox3','bbox4','bbox5','bbox6', ], nargs='+')
    parser.add_argument('--excel_file', type=str, default='')
    parser.add_argument('--csv_output_dir', type=str, default='csv_outputs')
    parser.add_argument('--checkpoint_dir', type=str, default='checkpoints')
    parser.add_argument('--log_dir', type=str, default='logs')
    parser.add_argument('--size', type=tuple, default=(80,80))
    parser.add_argument('--batch_size', type=int, default=2)
    parser.add_argument('--num_epochs', type=int, default=5)
    parser.add_argument('--learning_rate', type=float, default=1e-4)
    parser.add_argument('--weight_decay', type=float, default=1e-2)
    parser.add_argument('--subvol_size', type=int, default=80)
    parser.add_argument('--num_frames', type=int, default=80)
    parser.add_argument('--mask_ratio', type=float, default=0.75)
    parser.add_argument('--patience', type=int, default=3)
    parser.add_argument('--resume_checkpoint', type=str, default=None)
    parser.add_argument('--save_gifs_dir', type=str, default='gifs')
    parser.add_argument('--num_gifs', type=int, default=10)
    parser.add_argument('--alpha', type=float, default=0.3)
    parser.add_argument('--segmentation_type', type=int, default=0, choices=range(0, 6))
    parser.add_argument('--input_mask', action='store_true', # Added argument
                       help='Mask input image using segmentation_type regions')
    args, _ = parser.parse_known_args()

    return args

def main(args):
    processor = SimpleVideoProcessor(size=(80, 80))
    vol_data_dict = {}

    for bbox_name in args.bbox_name:
        raw_vol, seg_vol, add_mask_vol = load_volumes(
            bbox_name=bbox_name,
            raw_base_dir=args.raw_base_dir,
            seg_base_dir=args.seg_base_dir,
            add_mask_base_dir=args.add_mask_base_dir
        )
        if raw_vol is not None:
            vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

    synapse_dfs = []
    for bbox_name in args.bbox_name:
        excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
        if os.path.exists(excel_path):
            df = pd.read_excel(excel_path)
            df['bbox_name'] = bbox_name
            synapse_dfs.append(df)

    syn_df = pd.concat(synapse_dfs, ignore_index=True)
    dataset = VideoMAEDataset(
        vol_data_dict=vol_data_dict,
        synapse_df=syn_df,
        processor=processor,
        segmentation_type=args.segmentation_type,
        subvol_size=args.subvol_size,
        num_frames=args.num_frames,
        alpha=args.alpha,
        input_mask=args.input_mask  # Pass the flag
    )

    cubes = []
    syn_info_list = []
    for idx in range(len(dataset)):
        pixel_values, syn_info, _ = dataset[idx]
        cubes.append(pixel_values)
        syn_info_list.append(syn_info)

    print(f"Processed {len(cubes)} cubes successfully.")
    return cubes, pd.DataFrame(syn_info_list)

# if __name__ == "__main__":
#     args = parse_args()
#     cubes, sys_inf = main(args)
#     print(f"Final output: {len(cubes)} cubes")


# import torch
# import numpy as np
# import imageio

# cube = cubes[0]
# mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
# std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

# denormalized_cube = cube * std + mean

# denormalized_cube = torch.clamp(denormalized_cube, 0, 1)
# frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)
# frames = (frames * 255).astype(np.uint8)  # Convert to 0-255

# imageio.mimsave('synapse_cube.gif', frames, fps=10)

# print("GIF saved successfully!")



## 1.2 Test Average (sections that have not seg. be avg pix)

In [12]:
import os
import glob
import io
import argparse
import multiprocessing
from typing import List, Tuple

import numpy as np
import pandas as pd
import imageio.v3 as iio
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
class SimpleVideoProcessor:
    def __init__(self, size=(80, 80), mean=(0.485,),  # Use a single channel for grayscale
                 std=(0.229,)):
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(size),
            transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
            transforms.ToTensor(),
            transforms.Normalize(mean=mean, std=std),
        ])

    def __call__(self, frames, return_tensors=None):
        processed_frames = [self.transform(frame) for frame in frames]
        pixel_values = torch.stack(processed_frames)
        if return_tensors == "pt":
            return {"pixel_values": pixel_values}
        else:
            return pixel_values


def load_volumes(bbox_name: str, raw_base_dir: str, seg_base_dir: str, add_mask_base_dir: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    raw_dir = os.path.join(raw_base_dir, bbox_name)
    seg_dir = os.path.join(seg_base_dir, bbox_name)

    if bbox_name.startswith("bbox"):
        bbox_num = bbox_name.replace("bbox", "")
        add_mask_dir = os.path.join(add_mask_base_dir, f"bbox_{bbox_num}")
    else:
        add_mask_dir = os.path.join(add_mask_base_dir, bbox_name)

    raw_tif_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
    seg_tif_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))
    add_mask_tif_files = sorted(glob.glob(os.path.join(add_mask_dir, 'slice_*.tif')))

    if not (len(raw_tif_files) == len(seg_tif_files) == len(add_mask_tif_files)):
        return None, None, None

    try:
        raw_vol = np.stack([iio.imread(f) for f in raw_tif_files], axis=0)
        seg_vol = np.stack([iio.imread(f).astype(np.uint32) for f in seg_tif_files], axis=0)
        add_mask_vol = np.stack([iio.imread(f).astype(np.uint32) for f in add_mask_tif_files], axis=0)
        return raw_vol, seg_vol, add_mask_vol
    except Exception as e:
        return None, None, None
def create_segmented_cube(
    raw_vol: np.ndarray,
    seg_vol: np.ndarray,
    add_mask_vol: np.ndarray,
    central_coord: Tuple[int, int, int],
    side1_coord: Tuple[int, int, int],
    side2_coord: Tuple[int, int, int],
    segmentation_type: int,
    subvolume_size: int = 80,
    alpha: float = 0.3,
    input_mask: bool = False  # New parameter
) -> np.ndarray:
    def create_segment_masks(segmentation_volume, s1_coord, s2_coord):
        x1, y1, z1 = s1_coord
        x2, y2, z2 = s2_coord
        seg_id_1 = segmentation_volume[z1, y1, x1]
        seg_id_2 = segmentation_volume[z2, y2, x2]

        mask_1 = (segmentation_volume == seg_id_1) if seg_id_1 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
        mask_2 = (segmentation_volume == seg_id_2) if seg_id_2 != 0 else np.zeros_like(segmentation_volume, dtype=bool)
        return mask_1, mask_2

    mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)
    mask_3_full = (add_mask_vol > 0)

    half_size = subvolume_size // 2
    cx, cy, cz = central_coord
    x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
    y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
    z_start, z_end = max(cz - half_size, 0), min(cz + half_size, raw_vol.shape[0])

    sub_raw = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_3 = mask_3_full[z_start:z_end, y_start:y_end, x_start:x_end]

    pad_z = subvolume_size - sub_raw.shape[0]
    pad_y = subvolume_size - sub_raw.shape[1]
    pad_x = subvolume_size - sub_raw.shape[2]

    if pad_z > 0 or pad_y > 0 or pad_x > 0:
        sub_raw = np.pad(sub_raw, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=0)
        sub_mask_1 = np.pad(sub_mask_1, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_2 = np.pad(sub_mask_2, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_3 = np.pad(sub_mask_3, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)

    sub_raw = sub_raw[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_1 = sub_mask_1[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_2 = sub_mask_2[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_3 = sub_mask_3[:subvolume_size, :subvolume_size, :subvolume_size]

    overlaid_cube = np.zeros((subvolume_size, subvolume_size, 3, subvolume_size), dtype=np.uint8)

    for z in range(subvolume_size):
        raw_slice = sub_raw[z].astype(np.float32)
        mn, mx = raw_slice.min(), raw_slice.max()
        if mx > mn:
            raw_slice = (raw_slice - mn) / (mx - mn)
        else:
            raw_slice = raw_slice - mn

        if input_mask:  # New masking logic with average pixel filling
            combined_mask = np.zeros_like(raw_slice, dtype=np.float32)
            if segmentation_type in [1, 3, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_1[z])
            if segmentation_type in [2, 3, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_2[z])
            if segmentation_type in [4, 5]:
                combined_mask = np.logical_or(combined_mask, sub_mask_3[z])

            # Fill the non-segmented regions with the average of the segmented pixels
            if np.any(~combined_mask):  # Non-masked areas
                avg_value = np.mean(raw_slice[combined_mask == 0])  # Average of segmented pixels
                raw_slice[combined_mask == 0] = avg_value  # Replace non-segmented areas with the average

            # Convert to RGB (average pixel value in all channels)
            masked_rgb = np.stack([raw_slice] * 3, axis=-1)
            overlaid_image = (masked_rgb * 255).astype(np.uint8)
        else:  # Original overlay logic
            raw_rgb = np.stack([raw_slice] * 3, axis=-1)
            mask1_rgb = np.zeros_like(raw_rgb)
            mask2_rgb = np.zeros_like(raw_rgb)
            mask3_rgb = np.zeros_like(raw_rgb)

            if segmentation_type in [1, 3, 5]:
                mask1_rgb[sub_mask_1[z]] = [1, 0, 0]
            if segmentation_type in [2, 3, 5]:
                mask2_rgb[sub_mask_2[z]] = [0, 0, 1]
            if segmentation_type in [4, 5]:
                mask3_rgb[sub_mask_3[z]] = [0, 1, 0]

            combined_masks = mask1_rgb + mask2_rgb + mask3_rgb
            combined_masks = np.clip(combined_masks, 0, 1)
            overlaid_image = (1 - alpha) * raw_rgb + alpha * combined_masks
            overlaid_image = (np.clip(overlaid_image, 0, 1) * 255).astype(np.uint8)

        overlaid_cube[:, :, :, z] = overlaid_image

    return overlaid_cube


class VideoMAEDataset(Dataset):
    def __init__(self, vol_data_dict: dict, synapse_df: pd.DataFrame, processor,
                 segmentation_type: int, subvol_size: int = 80, num_frames: int = 16,
                 alpha: float = 0.3, input_mask: bool = False):  # Added input_mask
        self.vol_data_dict = vol_data_dict
        self.synapse_df = synapse_df.reset_index(drop=True)
        self.processor = processor
        self.segmentation_type = segmentation_type
        self.subvol_size = subvol_size
        self.num_frames = num_frames
        self.alpha = alpha
        self.input_mask = input_mask  # Store input_mask flag

    def __len__(self):
        return len(self.synapse_df)

    def __getitem__(self, idx):
        syn_info = self.synapse_df.iloc[idx]
        bbox_name = syn_info['bbox_name']
        raw_vol, seg_vol, add_mask_vol = self.vol_data_dict.get(bbox_name, (None, None, None))

        if raw_vol is None:
            return torch.zeros((self.num_frames, 3, self.subvol_size, self.subvol_size), dtype=torch.float32), syn_info, bbox_name

        central_coord = (int(syn_info['central_coord_1']), int(syn_info['central_coord_2']), int(syn_info['central_coord_3']))
        side1_coord = (int(syn_info['side_1_coord_1']), int(syn_info['side_1_coord_2']), int(syn_info['side_1_coord_3']))
        side2_coord = (int(syn_info['side_2_coord_1']), int(syn_info['side_2_coord_2']), int(syn_info['side_2_coord_3']))

        overlaid_cube = create_segmented_cube(
            raw_vol=raw_vol,
            seg_vol=seg_vol,
            add_mask_vol=add_mask_vol,
            central_coord=central_coord,
            side1_coord=side1_coord,
            side2_coord=side2_coord,
            segmentation_type=self.segmentation_type,
            subvolume_size=self.subvol_size,
            alpha=self.alpha,
            input_mask=self.input_mask  # Pass the flag
        )

        frames = [overlaid_cube[..., z] for z in range(overlaid_cube.shape[3])]
        if len(frames) < self.num_frames:
            frames += [frames[-1]] * (self.num_frames - len(frames))
        elif len(frames) > self.num_frames:
            indices = np.linspace(0, len(frames)-1, self.num_frames, dtype=int)
            frames = [frames[i] for i in indices]

        inputs = self.processor(frames, return_tensors="pt")
        return inputs["pixel_values"].squeeze(0).float(), syn_info, bbox_name

def parse_args():
    parser = argparse.ArgumentParser(description="VideoMAE Pre-training Script with Segmented Videos and Additional Masks")
    parser.add_argument('--raw_base_dir', type=str, default='raw')
    parser.add_argument('--seg_base_dir', type=str, default='seg')
    parser.add_argument('--add_mask_base_dir', type=str, default='')
    parser.add_argument('--bbox_name', type=str, default=['bbox1','bbox2','bbox3','bbox4','bbox5','bbox6', ], nargs='+')
    parser.add_argument('--excel_file', type=str, default='')
    parser.add_argument('--csv_output_dir', type=str, default='csv_outputs')
    parser.add_argument('--checkpoint_dir', type=str, default='checkpoints')
    parser.add_argument('--log_dir', type=str, default='logs')
    parser.add_argument('--size', type=tuple, default=(80,80))
    parser.add_argument('--batch_size', type=int, default=2)
    parser.add_argument('--num_epochs', type=int, default=5)
    parser.add_argument('--learning_rate', type=float, default=1e-4)
    parser.add_argument('--weight_decay', type=float, default=1e-2)
    parser.add_argument('--subvol_size', type=int, default=80)
    parser.add_argument('--num_frames', type=int, default=80)
    parser.add_argument('--mask_ratio', type=float, default=0.75)
    parser.add_argument('--patience', type=int, default=3)
    parser.add_argument('--resume_checkpoint', type=str, default=None)
    parser.add_argument('--save_gifs_dir', type=str, default='gifs')
    parser.add_argument('--num_gifs', type=int, default=10)
    parser.add_argument('--alpha', type=float, default=0.3)
    parser.add_argument('--segmentation_type', type=int, default=5, choices=range(0, 6))
    parser.add_argument('--input_mask', action='store_true', # Added argument
                       help='Mask input image using segmentation_type regions')
    args, _ = parser.parse_known_args()

    return args

def main(args):
    processor = SimpleVideoProcessor(size=(80, 80))
    vol_data_dict = {}

    for bbox_name in args.bbox_name:
        raw_vol, seg_vol, add_mask_vol = load_volumes(
            bbox_name=bbox_name,
            raw_base_dir=args.raw_base_dir,
            seg_base_dir=args.seg_base_dir,
            add_mask_base_dir=args.add_mask_base_dir
        )
        if raw_vol is not None:
            vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

    synapse_dfs = []
    for bbox_name in args.bbox_name:
        excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
        if os.path.exists(excel_path):
            df = pd.read_excel(excel_path)
            df['bbox_name'] = bbox_name
            synapse_dfs.append(df)

    syn_df = pd.concat(synapse_dfs, ignore_index=True)
    dataset = VideoMAEDataset(
        vol_data_dict=vol_data_dict,
        synapse_df=syn_df,
        processor=processor,
        segmentation_type=args.segmentation_type,
        subvol_size=args.subvol_size,
        num_frames=args.num_frames,
        alpha=args.alpha,
        input_mask=args.input_mask  # Pass the flag
    )

    cubes = []
    syn_info_list = []
    for idx in range(len(dataset)):
        pixel_values, syn_info, _ = dataset[idx]
        cubes.append(pixel_values)
        syn_info_list.append(syn_info)

    print(f"Processed {len(cubes)} cubes successfully.")
    return cubes, pd.DataFrame(syn_info_list)

# if __name__ == "__main__":
#     args = parse_args()
#     args.input_mask=True
#     cubes, sys_inf = main(args)
#     print(f"Final output: {len(cubes)} cubes")

# # Visualizeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
# import torch
# import numpy as np
# import imageio

# cube = cubes[0]
# mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
# std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)

# denormalized_cube = cube * std + mean

# denormalized_cube = torch.clamp(denormalized_cube, 0, 1)
# frames = denormalized_cube.permute(0, 2, 3, 1).numpy()  # Change to (T, H, W, C)
# frames = (frames * 255).astype(np.uint8)  # Convert to 0-255

# imageio.mimsave('synapse_cube.gif', frames, fps=10)

# print("GIF saved successfully!")



# VGG Model and Feature Extracture

In [5]:

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import pandas as pd
import os

class Vgg3D(nn.Module):
    def __init__(
        self,
        input_size=(80, 80, 80),
        fmaps=24,
        downsample_factors=[(2, 2, 2), (2, 2, 2), (2, 2, 2), (2, 2, 2)],
        fmap_inc=(2, 2, 2, 2),
        n_convolutions=(4, 2, 2, 2),
        output_classes=7,
        input_fmaps=3,
    ):
        super(Vgg3D, self).__init__()

        # Validate input parameters
        if len(downsample_factors) != len(fmap_inc):
            raise ValueError("fmap_inc needs to have same length as downsample factors")
        if len(n_convolutions) != len(fmap_inc):
            raise ValueError("n_convolutions needs to have the same length as downsample factors")
        if np.any(np.array(n_convolutions) < 1):
            raise ValueError("Each layer must have at least one convolution")

        current_fmaps = input_fmaps
        current_size = np.array(input_size)

        # Feature extraction layers
        layers = []
        for i, (df, nc) in enumerate(zip(downsample_factors, n_convolutions)):
            # Convolution block
            layers += [
                nn.Conv3d(current_fmaps, fmaps, kernel_size=3, padding=1),
                nn.BatchNorm3d(fmaps),
                nn.ReLU(inplace=True)
            ]

            # Additional convolutions
            for _ in range(nc - 1):
                layers += [
                    nn.Conv3d(fmaps, fmaps, kernel_size=3, padding=1),
                    nn.BatchNorm3d(fmaps),
                    nn.ReLU(inplace=True)
                ]

            # Downsampling
            layers.append(nn.MaxPool3d(df))

            # Update feature map size
            current_fmaps = fmaps
            fmaps *= fmap_inc[i]

            # Update spatial dimensions
            current_size = np.floor(current_size / np.array(df))
            # logger.info(f"Block {i+1}: features {current_fmaps}, size {current_size}")

        self.features = nn.Sequential(*layers)

        # Classifier (not used for feature extraction)
        self.classifier = nn.Sequential(
            nn.Linear(int(np.prod(current_size)) * current_fmaps, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, output_classes),
        )

    def forward(self, x, return_features=False):
        x = self.features(x)
        if return_features:
            return x  # Return raw features before flattening
        x = x.view(x.size(0), -1)
        return self.classifier(x)


def extract_features(model, dataset, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()

    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=2,
        collate_fn=lambda b: (
            torch.stack([item[0] for item in b]),  # Pixel values
            [item[1] for item in b],               # Synapse info
            [item[2] for item in b]                # Bbox names
        )
    )

    features = []
    metadata = []

    with torch.no_grad():
        for batch in dataloader:
            pixels, info, names = batch
            inputs = pixels.permute(0, 2, 1, 3, 4).to(device)  # Reshape for 3D convolution

            batch_features = model.features(inputs)
            pooled_features = nn.AdaptiveAvgPool3d((1, 1, 1))(batch_features)

            # Flatten and convert to numpy immediately
            features.append(pooled_features.cpu().numpy().squeeze())
            metadata.extend(zip(names, info))

    # Combine all batch features
    features = np.concatenate(features, axis=0)

    # Create metadata DataFrame
    metadata_df = pd.DataFrame([
        {"bbox": name, **info.to_dict()}
        for name, info in metadata
    ])

    # Create feature columns
    feature_columns = [f'feat_{i+1}' for i in range(features.shape[1])]
    features_df = pd.DataFrame(features, columns=feature_columns)

    # Combine with metadata
    combined_df = pd.concat([metadata_df, features_df], axis=1)

    return combined_df
# # Initialize the model
# model = Vgg3D(input_size=(80, 80, 80), fmaps=24, downsample_factors=[(2, 2, 2), (2, 2, 2)],
#               fmap_inc=(2, 2), n_convolutions=(4, 2), output_classes=7, input_fmaps=1)

# Load model from checkpoint
def load_model_from_checkpoint(model, checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)

    # Load the state_dict into the model
    model.load_state_dict(checkpoint['model_state_dict'])

    # If the checkpoint includes the optimizer state, you can load that as well
    # model.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])  # if needed

    # Set the model to evaluation mode (disable dropout, batch norm updates, etc.)
    model.eval()

    return model


# Example: Load your model with the checkpoint
model = Vgg3D(input_size=(80, 80, 80), fmaps=24,output_classes=7, input_fmaps=1)

# Paths and directories
checkpoint_url = "https://dl.dropboxusercontent.com/scl/fo/mfejaomhu43aa6oqs6zsf/AKMAAgT7OrUtruR0AQXZBy0/hemibrain_production.checkpoint.20220225?rlkey=6cmwxdvehy4ylztvsbgkfnrfc&dl=0"
checkpoint_path = 'hemibrain_production.checkpoint'

# Download the checkpoint if it doesn't exist
if not os.path.exists(checkpoint_path):
    os.system(f"wget -O {checkpoint_path} '{checkpoint_url}'")
    print("Downloaded VGG3D checkpoint.")
else:
    print("VGG3D checkpoint already exists.")


checkpoint_path = 'hemibrain_production.checkpoint'  # Replace with the actual path to your checkpoint
model = load_model_from_checkpoint(model, checkpoint_path)


Downloaded VGG3D checkpoint.


<ipython-input-5-16434c063c55>:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


In [6]:
!mkdir Result


#raw

In [7]:

args = parse_args()
args.input_mask=False
args.segmentation_type=0
# Initialize the arguments (or use your existing args)
# Use the main function to load data and prepare the dataset
# cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=args.segmentation_type,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=False
)

# Now extract features from the model
features_df = extract_features(model, dataset, args)

# Print the resulting DataFrame
# print(features_df.head())
import os
import torch
import numpy as np
import imageio

# Extract the data tensor (cube) from the tuple returned by dataset[0]
cube, syn_info, bbox_name = dataset[0]

mean = torch.tensor([0.485]).view(1, 1, 1, 1)
std = torch.tensor([0.229]).view(1, 1, 1, 1)

denormalized_cube = cube * std + mean
denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# Convert to RGB if needed
frames = denormalized_cube.squeeze(1).numpy()  # Remove channel dimension
frames = (frames * 255).astype(np.uint8)

# Stack to create RGB
frames = np.stack([frames, frames, frames], axis=-1)

Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_True"
output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# Save the frames as a gif using imageio
imageio.mimsave(output_gif_path, frames, fps=10)

print("GIF saved successfully!")

segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_True"
output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")

# Save the features_df DataFrame as a CSV file
features_df.to_csv(output_csv_path, index=False)

print(f"Features saved successfully as {output_csv_path}")

# Set device

# Example of processing with Grad-CAM



GIF saved successfully!
Features saved successfully as Result/VGG_CSV_segmentation_type_0_input_mask_False_features.csv


In [49]:

# # # Extract the data tensor (cube) from the tuple returned by dataset[0]
# # cube, syn_info, bbox_name = dataset[0]

# # # Define the mean and std values for denormalization
# # mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB
# # std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)  # 1x3x1x1 for RGB

# mean = torch.tensor([0.485]).view(1, 1, 1, 1)
# std = torch.tensor([0.229]).view(1, 1, 1, 1)

# denormalized_cube = cube * std + mean
# denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# # Convert to RGB if needed
# frames = denormalized_cube.squeeze(1).numpy()  # Remove channel dimension
# frames = (frames * 255).astype(np.uint8)

# # Stack to create RGB
# frames = np.stack([frames, frames, frames], axis=-1)

# Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# # Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_True"
# output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# # Save the frames as a gif using imageio
# imageio.mimsave(output_gif_path, frames, fps=10)

# print("GIF saved successfully!")

# segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# # segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_True"
# output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")

# # Save the features_df DataFrame as a CSV file
# features_df.to_csv(output_csv_path, index=False)

# print(f"Features saved successfully as {output_csv_path}")

# # Set device

# # Example of processing with Grad-CAM

GIF saved successfully!
Features saved successfully as Result/VGG_CSV_segmentation_type_0_input_mask_False_features.csv


In [15]:
# Print the layers of the Vgg3D model
def print_model_layers(model):
    for name, module in model.named_modules():
        print(f"{name}: {module}")

print_model_layers(model)


: Vgg3D(
  (features): Sequential(
    (0): Conv3d(1, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (4): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (7): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (10): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (13): Conv3d(24, 48, kernel_size=(3, 3, 3), stride=(1, 

In [12]:
process_with_grad_cam(model, dataset, device)


RuntimeError: Given groups=1, weight of size [24, 1, 3, 3, 3], expected input[1, 80, 1, 80, 80] to have 1 channels, but got 80 channels instead

# Seg 5 False

In [30]:

# checkpoint_path = 'hemibrain_production.checkpoint'  # Replace with the actual path to your checkpoint
# model = load_model_from_checkpoint(model, checkpoint_path)

# Initialize the arguments (or use your existing args)
args = parse_args()

args.input_mask=False
args.segmentation_type=5
# Use the main function to load data and prepare the dataset
cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=5,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=False
)

# Now extract features from the model
features_df = extract_features(model, dataset, args)

# Print the resulting DataFrame
# print(features_df.head())
import os
import torch
import numpy as np
import imageio

# Extract the data tensor (cube) from the tuple returned by dataset[0]
cube, syn_info, bbox_name = dataset[0]

mean = torch.tensor([0.485]).view(1, 1, 1, 1)
std = torch.tensor([0.229]).view(1, 1, 1, 1)

denormalized_cube = cube * std + mean
denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# Convert to RGB if needed
frames = denormalized_cube.squeeze(1).numpy()  # Remove channel dimension
frames = (frames * 255).astype(np.uint8)

# Stack to create RGB
frames = np.stack([frames, frames, frames], axis=-1)

Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# Gif_Name = "VGG_Gif_segmentation_type_5_input_mask_False"
output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# Save the frames as a gif using imageio
imageio.mimsave(output_gif_path, frames, fps=10)

print("GIF saved successfully!")

segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# segmentation_type_name = "VGG_CSV_segmentation_type_5_input_mask_False"
output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")

# Save the features_df DataFrame as a CSV file
features_df.to_csv(output_csv_path, index=False)

print(f"Features saved successfully as {output_csv_path}")


Processed 444 cubes successfully.


RuntimeError: cannot register a hook on a tensor that doesn't require gradient

# Seg5 True Black

In [51]:
args = parse_args()

args.input_mask=True
args.segmentation_type=5
# Initialize the arguments (or use your existing args)

# Use the main function to load data and prepare the dataset
# cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=5,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=True
)

# Now extract features from the model
features_df = extract_features(model, dataset, args)

# Print the resulting DataFrame
# print(features_df.head())
import os
import torch
import numpy as np
import imageio

# Extract the data tensor (cube) from the tuple returned by dataset[0]
cube, syn_info, bbox_name = dataset[0]

mean = torch.tensor([0.485]).view(1, 1, 1, 1)
std = torch.tensor([0.229]).view(1, 1, 1, 1)

denormalized_cube = cube * std + mean
denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# Convert to RGB if needed
frames = denormalized_cube.squeeze(1).numpy()  # Remove channel dimension
frames = (frames * 255).astype(np.uint8)

# Stack to create RGB
frames = np.stack([frames, frames, frames], axis=-1)

Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
# Gif_Name = "VGG_Gif_segmentation_type_5_input_mask_True"
output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# Save the frames as a gif using imageio
imageio.mimsave(output_gif_path, frames, fps=10)

print("GIF saved successfully!")

segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}_Black"
# segmentation_type_name = "VGG_CSV_segmentation_type_5_input_mask_True"
output_csv_path = os.path.join("Result", f"{segmentation_type_name}_Black_features.csv")

# Save the features_df DataFrame as a CSV file
features_df.to_csv(output_csv_path, index=False)

print(f"Features saved successfully as {output_csv_path}")


GIF saved successfully!
Features saved successfully as Result/VGG_CSV_segmentation_type_5_input_mask_True_Black_Black_features.csv


# seg5 True Avg

## Before run this run 1.2

In [13]:

# checkpoint_path = 'hemibrain_production.checkpoint'  # Replace with the actual path to your checkpoint
# model = load_model_from_checkpoint(model, checkpoint_path)

# Initialize the arguments (or use your existing args)
args = parse_args()

args.input_mask=True
args.segmentation_type=5
# Use the main function to load data and prepare the dataset
# cubes, sys_inf = main(args)

# Define the processor (if not already done)
processor = SimpleVideoProcessor(size=(80, 80))

# Prepare the dataset
vol_data_dict = {}
for bbox_name in args.bbox_name:
    raw_vol, seg_vol, add_mask_vol = load_volumes(
        bbox_name=bbox_name,
        raw_base_dir=args.raw_base_dir,
        seg_base_dir=args.seg_base_dir,
        add_mask_base_dir=args.add_mask_base_dir
    )
    if raw_vol is not None:
        vol_data_dict[bbox_name] = (raw_vol, seg_vol, add_mask_vol)

synapse_dfs = []
for bbox_name in args.bbox_name:
    excel_path = os.path.join(args.excel_file, f"{bbox_name}.xlsx")
    if os.path.exists(excel_path):
        df = pd.read_excel(excel_path)
        df['bbox_name'] = bbox_name
        synapse_dfs.append(df)

syn_df = pd.concat(synapse_dfs, ignore_index=True)

# Prepare the dataset
dataset = VideoMAEDataset(
    vol_data_dict=vol_data_dict,
    synapse_df=syn_df,
    processor=processor,
    segmentation_type=5,
    subvol_size=args.subvol_size,
    num_frames=args.num_frames,
    alpha=args.alpha,
    input_mask=True
)

# Now extract features from the model
features_df = extract_features(model, dataset, args)

# Print the resulting DataFrame
# print(features_df.head())
import os
import torch
import numpy as np
import imageio

# Extract the data tensor (cube) from the tuple returned by dataset[0]
cube, syn_info, bbox_name = dataset[0]

mean = torch.tensor([0.485]).view(1, 1, 1, 1)
std = torch.tensor([0.229]).view(1, 1, 1, 1)

denormalized_cube = cube * std + mean
denormalized_cube = torch.clamp(denormalized_cube, 0, 1)

# Convert to RGB if needed
frames = denormalized_cube.squeeze(1).numpy()  # Remove channel dimension
frames = (frames * 255).astype(np.uint8)

# Stack to create RGB
frames = np.stack([frames, frames, frames], axis=-1)

# Gif_Name = f"VGG_Gif_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
Gif_Name = "VGG_Gif_segmentation_type_5_input_mask_True_AVG"
output_gif_path = os.path.join("Result", f"{Gif_Name}.gif")
# Save the frames as a gif using imageio
imageio.mimsave(output_gif_path, frames, fps=10)

print("GIF saved successfully!")

# segmentation_type_name = f"VGG_CSV_segmentation_type_{args.segmentation_type}_input_mask_{args.input_mask}"
segmentation_type_name = "VGG_CSV_segmentation_type_5_input_mask_True_AVG"
output_csv_path = os.path.join("Result", f"{segmentation_type_name}_features.csv")

# Save the features_df DataFrame as a CSV file
features_df.to_csv(output_csv_path, index=False)

print(f"Features saved successfully as {output_csv_path}")


GIF saved successfully!
Features saved successfully as Result/VGG_CSV_segmentation_type_5_input_mask_True_AVG_features.csv


In [17]:
import shutil

# Replace 'folder_name' with the name of your folder
shutil.make_archive('/content/Result', 'zip', '/content/Result')
from google.colab import files

# Download the zip file
files.download('/content/Result.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
!dir

bbox_1	    bbox_7
bbox1.xlsx  bbox7.xlsx
bbox_2	    downloaded_file.zip
bbox2.xlsx  hemibrain_production.checkpoint
bbox_3	    __MACOSX
bbox3.xlsx  raw
bbox_4	    Result
bbox4.xlsx  Result.zip
bbox_5	    sample_data
bbox5.xlsx  seg
bbox_6	    vesicle_cloud__syn_interface__mitochondria_annotation.zip
bbox6.xlsx


# Classification

In [19]:
def classify(model, dataset, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()

    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=2,
        collate_fn=lambda b: (
            torch.stack([item[0] for item in b]),  # Pixel values
            [item[1] for item in b],               # Synapse info
            [item[2] for item in b]                # Bbox names
        )
    )

    all_preds = []
    metadata = []

    with torch.no_grad():
        for batch in dataloader:
            pixels, info, names = batch
            inputs = pixels.permute(0, 2, 1, 3, 4).to(device)  # [batch, channels, depth, H, W]

            # Get model predictions
            outputs = model(inputs)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds.append(preds.cpu().numpy())
            metadata.extend(zip(names, info))

    # Combine results
    all_preds = np.concatenate(all_preds, axis=0)

    # Create DataFrame
    results_df = pd.DataFrame([
        {"bbox": name, **info.to_dict(), "predicted_class": pred}
        for (name, info), pred in zip(metadata, all_preds)
    ])

    return results_df

In [22]:
res=classify(model, dataset,args)

In [23]:
res.head()

,bbox,Var1,central_coord_1,central_coord_2,central_coord_3,side_1_coord_1,side_1_coord_2,side_1_coord_3,side_2_coord_1,side_2_coord_2,side_2_coord_3,bbox_name,predicted_class
0,bbox1,non_spine_synapsed_056,171,260,350,171,268,359,171,260,340,bbox1,6
1,bbox1,non_spine_synapse_057,223,113,425,223,112,438,223,114,407,bbox1,6
2,bbox1,non_spine_synapse_058,280,102,377,280,94,400,280,108,364,bbox1,6
3,bbox1,non_spine_synapse_063,455,131,162,455,134,181,455,127,145,bbox1,6
4,bbox1,non_spine_synapse_062,138,121,302,135,113,298,140,127,312,bbox1,6


In [24]:
# prompt: save res at csv

res.to_csv('Result/classification_results.csv', index=False)

In [25]:
def classify_with_probs(model, dataset, args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()

    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=2,
        collate_fn=lambda b: (
            torch.stack([item[0] for item in b]),  # Pixel values
            [item[1] for item in b],               # Synapse info
            [item[2] for item in b]                # Bbox names
        )
    )

    all_preds = []
    all_probs = []
    metadata = []

    with torch.no_grad():
        for batch in dataloader:
            pixels, info, names = batch
            inputs = pixels.permute(0, 2, 1, 3, 4).to(device)  # [batch, channels, depth, H, W]

            # Get model predictions
            outputs = model(inputs)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            # Store results
            all_probs.append(probs.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
            metadata.extend(zip(names, info))

    # Combine results
    all_preds = np.concatenate(all_preds, axis=0)
    all_probs = np.concatenate(all_probs, axis=0)

    # Create DataFrame with probabilities
    results_df = pd.DataFrame([
        {
            "bbox": name,
            **info.to_dict(),
            "predicted_class": pred,
            **{f"prob_class_{i}": prob[i] for i in range(len(prob))}
        }
        for (name, info), pred, prob in zip(metadata, all_preds, all_probs)
    ])

    return results_df

# Run classification with probabilities
results_with_probs = classify_with_probs(model, dataset, args)

# Save full results
results_with_probs.to_csv("Result/classification_results_with_probabilities.csv", index=False)
print("Results with probabilities saved to Result/classification_results_with_probabilities.csv")

Results with probabilities saved to Result/classification_results_with_probabilities.csv
